# Further Analysis of Neurotransmitter Genes and Patient Survival

Now we can focus on determining the effect that gene expression from our various neurotransmitter genes of interest have on patient survival rates. 

We can start by importing our necessary data tables

## Part 0: Data Preparation

In [1]:
# importing data analysis libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# fixes the setting with copy warning
pd.options.mode.chained_assignment = None

In [3]:
def filter_survival_table(file_path):
    full_table = pd.read_csv('datasets/TCGA_PAAD_survival_data.csv', delimiter=',')
    survival_filtered = full_table[full_table.columns.intersection(['submitter_id', 'Disease Free Status',
                                              'Overall Survival (Months)', 'Overall Survival Status'])]
    
    return survival_filtered


In [4]:
survival = pd.read_csv('datasets/TCGA_PAAD_survival_data.csv', delimiter=',')

In [5]:
survival

,Cancer_type,submitter_id,PNI,LVI,Filename,Diagnosis Age,Sex,Race Category,Disease Free (Months),Disease Free Status,Overall Survival (Months),Overall Survival Status
0,PAAD,TCGA-HZ-7923,0,0,42bec5f7-7623-42e6-bbdf-514fe3805940.htseq.cou...,65,Male,WHITE,10.32,DiseaseFree,10.32,LIVING
1,PAAD,TCGA-US-A776,0,0,19a3b9bb-d4cb-4925-a87a-57f724141a67.htseq.cou...,61,Male,WHITE,39.95,DiseaseFree,39.95,LIVING
2,PAAD,TCGA-HZ-A77Q,0,0,03630a0c-aa97-4e28-bac9-0206fff669cd.htseq.cou...,55,Female,WHITE,1.08,DiseaseFree,1.08,LIVING
3,PAAD,TCGA-2J-AABA,0,0,c4cadcbb-ae87-43a5-811c-6bedda1d1d8d.htseq.cou...,55,Male,WHITE,15.51,Recurred/Progressed,19.94,DECEASED
4,PAAD,TCGA-F2-6879,0,0,16c63027-f745-41c4-a5e8-f6d9f1fbf1c8.htseq.cou...,57,Male,WHITE,6.01,Recurred/Progressed,10.97,DECEASED
...,...,...,...,...,...,...,...,...,...,...,...,...
149,PAAD,TCGA-2J-AABO,1,-,c642e018-f0cb-4be8-9b19-c944f1daf9cf.htseq.cou...,43,Male,WHITE,14.45,Recurred/Progressed,14.45,LIVING
150,PAAD,TCGA-H6-A45N,1,-,1f997074-0020-47e6-9928-5bf7209c552d.htseq.cou...,88,Female,WHITE,11.93,Recurred/Progressed,13.83,DECEASED
151,PAAD,TCGA-2L-AAQI,1,-,d5612378-33e0-4fe5-ad2f-a1887fb7b5cd.htseq.cou...,66,Male,WHITE,NaN,NaN,3.38,DECEASED
152,PAAD,TCGA-2L-AAQJ,1,-,d7d3fe8e-3885-44e8-934c-50f2a3bbfb2f.htseq.cou...,49,Female,WHITE,NaN,NaN,12.94,DECEASED


In [6]:
survival_filtered = filter_survival_table('datasets/TCGA_PAAD_survival_data.csv')

survival_filtered

,submitter_id,Disease Free Status,Overall Survival (Months),Overall Survival Status
0,TCGA-HZ-7923,DiseaseFree,10.32,LIVING
1,TCGA-US-A776,DiseaseFree,39.95,LIVING
2,TCGA-HZ-A77Q,DiseaseFree,1.08,LIVING
3,TCGA-2J-AABA,Recurred/Progressed,19.94,DECEASED
4,TCGA-F2-6879,Recurred/Progressed,10.97,DECEASED
...,...,...,...,...
149,TCGA-2J-AABO,Recurred/Progressed,14.45,LIVING
150,TCGA-H6-A45N,Recurred/Progressed,13.83,DECEASED
151,TCGA-2L-AAQI,NaN,3.38,DECEASED
152,TCGA-2L-AAQJ,NaN,12.94,DECEASED


Attempt 1 to filter rnaseq table

In [7]:
all_rnaseq = pd.read_csv('datasets/tpm_rnaseq_values.csv', delimiter = ',')
all_rnaseq.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
def create_counts_list(patient_table):
    
    count_files = patient_table['Filename'].tolist()
    
    count_files_list = []
    # the filenames column is formatted differently from the column names in the RNASeq table
    for file in count_files:
        remove_dash = file.replace('-', '.')
        #remove_tab = remove_dash.replace('\t', '')
        count_files_list.append(remove_dash)
    
    return count_files_list

In [9]:
def filter_patients(rnaseq_table):
    count_files = create_counts_list(survival)
    rnaseq_survival = all_rnaseq[all_rnaseq.columns.intersection(count_files)]
    first_column = rnaseq_table['hgnc_symbol']
    rnaseq_survival.insert(0, 'hgnc_symbol', first_column)
    
    return rnaseq_survival

In [10]:
rnaseq_survival = filter_patients(all_rnaseq)

In [11]:
rnaseq_survival

,hgnc_symbol,a0f5f7d4.88e0.4f3b.853b.e1e4f6bca748.htseq.counts.gz,a19219b1.db59.4493.83ef.e938e2ffdefd.htseq.counts.gz,a2a33be8.232b.44bf.a003.349017a5bc5a.htseq.counts.gz,a995d6ba.19c7.498f.b2d8.3f9b4b4826a1.htseq.counts.gz,aab761be.87c9.41a2.99b8.8bed9032333c.htseq.counts.gz,aec2e0c7.4792.41af.873c.3f3a53ec6d38.htseq.counts.gz,aeeb2bc3.c26d.4988.b4d6.425c7b2db8ae.htseq.counts.gz,b0159d01.f1eb.490d.875b.cfdabed6f529.htseq.counts.gz,b14ad58e.4740.4c4f.83ff.7535efe6a055.htseq.counts.gz,...,eb3894d4.fcae.43ef.ad68.b756c6aa56ea.htseq.counts.gz,f144de50.6126.4912.9c94.824d1eb0fac5.htseq.counts.gz,f2389819.b8fc.460e.821c.01dba313cce1.htseq.counts.gz,f6bd7191.a820.4d86.927a.b4b5f88ebd67.htseq.counts.gz,f748bf78.4dc1.47ad.8611.8186479d3e4b.htseq.counts.gz,f8551a29.d4bd.4954.bf9c.8e10265063de.htseq.counts.gz,f9f63982.b0ee.4cb8.8de5.f885d82137f0.htseq.counts.gz,fcd43085.7338.43fe.bc25.9d87b04e227f.htseq.counts.gz,feb22766.4282.47c8.bfe2.7d020b4a15d4.htseq.counts.gz,fef65b57.c58d.4050.8de4.f09f5cd616ce.htseq.counts.gz
0,A1BG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,A1BG-AS1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,A1CF,1.673631,1.077572,1.475284,1.691961,1.955460,2.487861,1.811539,2.517883,1.335224,...,1.840984,1.085729,1.219825,2.605873,1.391728,0.914085,2.117189,1.703033,1.026183,1.366619
3,A2M,0.065861,0.049196,0.037489,0.030524,0.030343,0.025105,0.038422,0.031410,0.060248,...,0.137374,0.033416,0.055014,0.052197,0.028828,0.052467,0.142909,0.053788,0.007786,0.087196
4,A2M-AS1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37333,ZYG11AP1,0.357896,0.259111,0.490654,0.076556,1.562104,0.033185,1.304941,0.166976,0.049109,...,0.682356,0.233470,0.230623,0.510561,0.920743,0.957988,6.032869,1.802480,0.179193,0.938191
37334,ZYG11B,2.914236,1.598769,0.812127,0.463809,0.676156,0.994286,0.935604,1.669530,1.508170,...,1.469878,0.791683,1.113077,1.503630,0.856166,1.399279,1.226022,0.688100,0.798788,1.181202
37335,ZYX,3.489280,2.063261,2.763898,3.717366,4.969874,14.381349,7.698801,72.471488,17.114665,...,8.134696,9.797582,2.881271,11.321482,5.061776,11.965091,10.241018,6.305561,3.145112,4.736807
37336,ZYXP1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.290379,0.000000,0.000000,0.000000,0.000000


tests came up a little weird, the count files are not exact matches

### Let's try to use the submitter_id to filter instead, using the PAAD patients file for filenames/count_files

In [12]:
def filter_by_submitter_id(rnaseq_table, patient_table):
    submitter_id_list = patient_table['submitter_id'].tolist()
    all_paad_patients = pd.read_csv('datasets/Sophia_PAAD_patients.csv', delimiter = ',')
    paad_patients = all_paad_patients[all_paad_patients['submitter_id'].isin(submitter_id_list)]
    # nothing was filtered, all submitter ids present in both tables
    counts = create_counts_list(paad_patients)
    
    first_column = rnaseq_table['hgnc_symbol']
    rnaseq_survival = all_rnaseq[all_rnaseq.columns.intersection(counts)]
    rnaseq_survival.insert(0, 'hgnc_symbol', first_column)
    return rnaseq_survival

rnaseq_survival_byid = filter_by_submitter_id(all_rnaseq, survival)

In [13]:
rnaseq_survival_byid

,hgnc_symbol,a0f5f7d4.88e0.4f3b.853b.e1e4f6bca748.htseq.counts.gz,a19219b1.db59.4493.83ef.e938e2ffdefd.htseq.counts.gz,a2a33be8.232b.44bf.a003.349017a5bc5a.htseq.counts.gz,a995d6ba.19c7.498f.b2d8.3f9b4b4826a1.htseq.counts.gz,aab761be.87c9.41a2.99b8.8bed9032333c.htseq.counts.gz,aec2e0c7.4792.41af.873c.3f3a53ec6d38.htseq.counts.gz,aeeb2bc3.c26d.4988.b4d6.425c7b2db8ae.htseq.counts.gz,b0159d01.f1eb.490d.875b.cfdabed6f529.htseq.counts.gz,b14ad58e.4740.4c4f.83ff.7535efe6a055.htseq.counts.gz,...,eb3894d4.fcae.43ef.ad68.b756c6aa56ea.htseq.counts.gz,f144de50.6126.4912.9c94.824d1eb0fac5.htseq.counts.gz,f2389819.b8fc.460e.821c.01dba313cce1.htseq.counts.gz,f6bd7191.a820.4d86.927a.b4b5f88ebd67.htseq.counts.gz,f748bf78.4dc1.47ad.8611.8186479d3e4b.htseq.counts.gz,f8551a29.d4bd.4954.bf9c.8e10265063de.htseq.counts.gz,f9f63982.b0ee.4cb8.8de5.f885d82137f0.htseq.counts.gz,fcd43085.7338.43fe.bc25.9d87b04e227f.htseq.counts.gz,feb22766.4282.47c8.bfe2.7d020b4a15d4.htseq.counts.gz,fef65b57.c58d.4050.8de4.f09f5cd616ce.htseq.counts.gz
0,A1BG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,A1BG-AS1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,A1CF,1.673631,1.077572,1.475284,1.691961,1.955460,2.487861,1.811539,2.517883,1.335224,...,1.840984,1.085729,1.219825,2.605873,1.391728,0.914085,2.117189,1.703033,1.026183,1.366619
3,A2M,0.065861,0.049196,0.037489,0.030524,0.030343,0.025105,0.038422,0.031410,0.060248,...,0.137374,0.033416,0.055014,0.052197,0.028828,0.052467,0.142909,0.053788,0.007786,0.087196
4,A2M-AS1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37333,ZYG11AP1,0.357896,0.259111,0.490654,0.076556,1.562104,0.033185,1.304941,0.166976,0.049109,...,0.682356,0.233470,0.230623,0.510561,0.920743,0.957988,6.032869,1.802480,0.179193,0.938191
37334,ZYG11B,2.914236,1.598769,0.812127,0.463809,0.676156,0.994286,0.935604,1.669530,1.508170,...,1.469878,0.791683,1.113077,1.503630,0.856166,1.399279,1.226022,0.688100,0.798788,1.181202
37335,ZYX,3.489280,2.063261,2.763898,3.717366,4.969874,14.381349,7.698801,72.471488,17.114665,...,8.134696,9.797582,2.881271,11.321482,5.061776,11.965091,10.241018,6.305561,3.145112,4.736807
37336,ZYXP1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.290379,0.000000,0.000000,0.000000,0.000000


so basically in the above portions, the first route we took was to pull Filenames directly out of the survival table, and then apply those Filenames onto the columns of our rnaseq table.

the second route was to pull submitter_id out of the survival table, lookup the Filenames from the submitter_id in the paad_patients file, and then filter the columns of the rnaseq table.

In [14]:
def difference_between_two_lists(list1, list2):
    return list(set(list1) - set(list2))

difference_between_two_lists(rnaseq_survival_byid.columns.values, rnaseq_survival.columns.values)

['cd1bba0f.b2e8.45e3.9b37.dcec5472cb7b.htseq.counts.gz']

# Part 1: Sorting and Filtering RNASeq Data

we can filter through our rnaseq data for patients of interest

In [39]:
def patient_percentile_by_gene(table, gene):
    
    copy = table.copy()
    gene_index = copy.index[copy['hgnc_symbol'] == gene].tolist()
    percentile = int(len(copy.columns.values) * 0.25)
    
    to_sort = copy.loc[copy['hgnc_symbol'] == gene]
    
    # for removing the hgnc column and adding back in later
    hgnc = to_sort.loc[:, 'hgnc_symbol']
    to_sort.drop('hgnc_symbol', axis=1, inplace=True)

    #return to_sort
    to_sort.sort_values(by=gene_index[0], ascending=False, axis=1, inplace=True)
    to_sort.insert(0, 'hgnc_symbol', hgnc)
    
    upper_quartile = to_sort.iloc[:, :percentile+1]
    lower_quartile = to_sort.iloc[:, len(copy.columns.values)-percentile:]
    lower_quartile.insert(0, 'hgnc_symbol', hgnc)
    
    return upper_quartile, lower_quartile

upper_quartile, lower_quartile = patient_percentile_by_gene(all_rnaseq, 'ADRB2')

In [40]:
upper_quartile

,hgnc_symbol,X6c73911d.8d0a.4a5a.9251.4ded7ea70fef.htseq.counts.gz,X98b1beb5.8d4c.45d1.a618.2d43aafa056c.htseq.counts.gz,X25b906db.fd54.4b85.b67e.e421826bd794.htseq.counts.gz,X08562fbc.4f41.408b.8bd1.0c656071d69f.htseq.counts.gz,d42d7f7b.1ba5.4ba5.8af2.35755fef2d69.htseq.counts.gz,cde0617f.fcd6.4c58.b3cf.dcfabc6fcda7.htseq.counts.gz,eb3894d4.fcae.43ef.ad68.b756c6aa56ea.htseq.counts.gz,X136067dc.fde9.42e4.8488.d38c7e2d713c.htseq.counts.gz,b9ab7393.4abb.41ec.9d55.a3dc846c4a93.htseq.counts.gz,...,X44c3d518.14fa.4d63.b265.d7fc81c398e2.htseq.counts.gz,X844163ae.2444.4406.9873.12e8295e3e07.htseq.counts.gz,b0159d01.f1eb.490d.875b.cfdabed6f529.htseq.counts.gz,X65a45bca.b5d4.4763.a51f.f7b9ad9efcb9.htseq.counts.gz,e8924790.8df6.40ed.be0d.2c74d8c2d974.htseq.counts.gz,cc133a8a.77ad.4570.852e.df72548c6897.htseq.counts.gz,X87c17703.72c7.4207.8935.3391f207f4a6.htseq.counts.gz,feb22766.4282.47c8.bfe2.7d020b4a15d4.htseq.counts.gz,X798e9f56.9964.4544.9d06.6a3aea20d0a6.htseq.counts.gz,c98f395b.003f.41b1.abff.38b6b227d64c.htseq.counts.gz
524,ADRB2,37.306216,27.620327,27.029471,24.999545,24.161432,23.958184,23.8802,23.605659,23.529877,...,19.801661,19.343447,19.047139,18.204514,18.029534,17.817788,17.759197,17.712482,17.023861,16.977867


In [41]:
lower_quartile

,hgnc_symbol,f6bd7191.a820.4d86.927a.b4b5f88ebd67.htseq.counts.gz,caf9cab4.f98f.46bd.a75d.0eb1e9c6c9ea.htseq.counts.gz,X59f31be1.8904.4395.a968.8bc9f68b5160.htseq.counts.gz,X7186ee30.db1a.43ad.955b.886cd5b5e771.htseq.counts.gz,a2a33be8.232b.44bf.a003.349017a5bc5a.htseq.counts.gz,e770b299.b4c2.4dee.95a8.abc3c5773439.htseq.counts.gz,X49895f4a.72ac.4d5e.ba56.8c8bb5de4758.htseq.counts.gz,X01f84c45.2058.4e22.b234.52f0a82a97fc.htseq.counts.gz,fef65b57.c58d.4050.8de4.f09f5cd616ce.htseq.counts.gz,...,X7989aed0.f0bc.4f0c.a8ae.f084221ed3f5.htseq.counts.gz,X30ce3471.112b.426c.ab57.6c0ab8e85dd2.htseq.counts.gz,X20f5f419.da3f.4c43.95fb.f7b6218c49f6.htseq.counts.gz,fcd43085.7338.43fe.bc25.9d87b04e227f.htseq.counts.gz,c642e018.f0cb.4be8.9b19.c944f1daf9cf.htseq.counts.gz,b490fdcb.d7ae.4473.93a5.eb332225b81b.htseq.counts.gz,X6423474d.60d7.4401.8e5b.46a3fbde5299.htseq.counts.gz,X232f085b.6201.4e4d.8473.e592b8d8e16d.htseq.counts.gz,X0be94b2f.fccb.4482.b0ea.695c101aa65a.htseq.counts.gz,a995d6ba.19c7.498f.b2d8.3f9b4b4826a1.htseq.counts.gz
524,ADRB2,10.001293,9.893382,9.676688,9.545395,9.118879,9.055446,8.960247,8.932134,8.808867,...,6.109053,6.036924,6.016945,5.460069,5.429504,5.250072,4.694696,4.490218,3.946579,3.787755
